In [1]:
# 3rd-Party Modules 
import music21
import tqdm
from keras.utils import np_utils
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Sequential
import pygame

# Built-In Modules 
import glob
import pickle

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


Based off this repo here: https://github.com/Skuldur/Classical-Piano-Composer

## Prepare the Data
Loop over our data that we have, converting them to music21 objects so we can pass it to our model. It takes a while so I've included a .pkl file that can be opened in the next step. 

In [2]:
notes = []

files = glob.glob("input_data/*/*.mid")
for x in tqdm.tqdm(range(len(files))):
    file = files[x]
    # Convert the file into a music21 objects
    midi = music21.converter.parse(file)
    
    # Variable to keep how many different notes will be needed to parse
    notes_to_parse = None
    
    # Seporate our any differnt parts if they exists
    parts = music21.instrument.partitionByInstrument(midi)
    if parts: 
        # File has differnt parts
        notes_to_parse = parts.parts[0].recurse()
    else: 
        # File does not have multiple parts
        notes_to_parse = midi.flat.notes
    
    # Loop over the notes we extracted
    for element in notes_to_parse:
        if isinstance(element, music21.note.Note):
            # If its a Note object -> add its pitch
            notes.append(str(element.pitch))
        elif isinstance(element, music21.chord.Chord):
            # Its its a Chord object -> Loop and add the ID of every note in the chord
            notes.append('.'.join(str(n) for n in element.normalOrder))

100%|██████████| 292/292 [02:29<00:00,  1.95it/s]


In [6]:
# Optional: Pickle the output, and save it for next time 
with open('checkpoints/notes_pickle.pkl', 'wb') as f:
    pickle.dump(notes, f)

## Process the data
Now we have our input data stored as a mix of words and numbers. ML always works better when working with numbers, so lets encode out data by creating a mapping from words to integers. 

In [3]:
# Optional: Open up the pickled file
with open('checkpoints/notes_pickle.pkl', 'rb') as f:
    notes = pickle.load(f)

In [15]:
# Number of 'vocab' words is the number of notes
n_vocab = len(set(notes))

# Variable sequence_length: This controls how much data is needed before a note, to predict the note itself
sequence_length = 100

# Grab all of the different pitch names and map the piches to integers
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

# Capture out input/output
network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

# Normalize the input (i.e. each value divded by the number of words)
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

## Create the Model

In [6]:
def create_model(network_input, n_vocab):
    """
    Creates the model that will be used to train the data 
    :param List network_input: Input data that will be used, needed for the .shape
    :param Integer n_vocab: The number of vocab words
    """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

model = create_model(network_input, n_vocab)

## Train Model

In [8]:
filepath = "checkpoints/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
4029/4029 [==============================] - 1610s 400ms/step - loss: 4.5204
Epoch 2/10
4029/4029 [==============================] - 1610s 400ms/step - loss: 4.4584
Epoch 3/10
4029/4029 [==============================] - 1610s 400ms/step - loss: 4.4025
Epoch 4/10
4029/4029 [==============================] - 1610s 400ms/step - loss: 4.2719
Epoch 5/10
4029/4029 [==============================] - 1610s 400ms/step - loss: 4.1391
Epoch 6/10
   4/4029 [..............................] - ETA: 20:03 - loss: 4.1043

KeyboardInterrupt: 

In [7]:
model = create_model(network_input, n_vocab)
model.load_weights("checkpoints/weights-improvement-05-4.1391-bigger.hdf5")

In [27]:
# pick a random sequence from the input as a starting point for the prediction
start = np.random.randint(0, len(network_input)-1)

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []

# generate 100 notes
for note_index in range(100):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern += index
    pattern = pattern[1:len(pattern)]


In [28]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = music21.note.Note(int(current_note))
                new_note.storedInstrument = music21.instrument.Piano()
                notes.append(new_note)
            new_chord = music21.chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = music21.note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = music21.instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

create_midi(prediction_output)

NameError: name 'note' is not defined

### Play the music created!
This is the fun part :). 

In [8]:
def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print("Music file %s loaded!" % music_file)
    except pygame.error:
        print("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
        
# pick a midi music file you have ...
# (if not in working folder use full path)
music_file = "test_output.mid"
freq = 44100    # audio CD quality
bitsize = -16   # unsigned 16 bit
channels = 1    # 1 is mono, 2 is stereo
buffer = 1024    # number of samples
pygame.mixer.init(freq, bitsize, channels, buffer)
# optional volume 0 to 1.0
pygame.mixer.music.set_volume(0.8)
try:
    play_music(music_file)
except KeyboardInterrupt:
    # if user hits Ctrl/C then exit
    # (works only in console mode)
    pygame.mixer.music.fadeout(1000)
    pygame.mixer.music.stop()
    raise SystemExit

Music file test_output.mid loaded!


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-8-35acf36524ff>", line 29, in <module>
    play_music(music_file)
  File "<ipython-input-8-35acf36524ff>", line 16, in play_music
    clock.tick(30)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sid/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-35acf36524ff>", line 35, in <module>
    raise SystemExit
SystemExit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sid/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1148, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/sid/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 316, in wrapped
    return f(*args, **kw

SystemExit: 

/home/sid/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
